In [1]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)

from sklearn.preprocessing import MinMaxScaler
import joblib

import sys
sys.path.insert(0, "../")
import functions

# <font size="7">Feature Selection</font>

<font size="5">Im Rahmen der Feature Selection werden die im Data Understanding als unrelevant beziehungsweise unbrauchbar ermittelten Daten entfernt. Übrig bleiben daher nur die aussagekräftigen und für die Modellierung brauchbaren Merkmale:</font>
<ul>
    <li><font size="5">Verbrauch</font></li>
    <li><font size="5">Arbeitstag</font></li>
    <li><font size="5">Temperatur</font></li>
    <li><font size="5">Sonnenaufgang</font></li>
    <li><font size="5">Sonnenuntergang</font></li>
</ul>

In [2]:
#Daten aus CSV-Datei laden
df = pd.read_csv("../2-Data Understanding/Datenbeschaffung/verbrauch.csv", index_col=0, usecols=[0, 1, 5], parse_dates=True)
df.index.freq = "D"

#Temperatur hinzufügen
df_ = pd.read_csv("../2-Data Understanding/Datenbeschaffung/kalender.csv", index_col=0, parse_dates=True, usecols=[0,2])
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/stuttgart.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("stuttgart"))
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/freiburg.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("freiburg"))
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/mannheim.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("mannheim"))
df_ = df_.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/ulm.csv", index_col=0, parse_dates=True, usecols=[0,2], squeeze=True).rename("ulm"))

df["temperatur"] = round(((df_["stuttgart"] + df_["freiburg"] + df_["mannheim"] + df_["ulm"]) / 4), 1)

#Sonnenauf-/Sonnenuntergang für Tagesstunden hinzufügen
df = df.join(pd.read_csv("../2-Data Understanding/Datenbeschaffung/stuttgart.csv", index_col=0, parse_dates=True, usecols=[0, 8, 9]))

#DataFrame ausgeben
print(df.head())
print()
print(df.describe().transpose())

            verbrauch  arbeitstag  temperatur sonnenaufgang sonnenuntergang
datum                                                                      
2015-01-01     126197           0        -2.5      08:16 AM        04:37 PM
2015-01-02     147085           1        -0.0      08:16 AM        04:38 PM
2015-01-03     141426           0         1.2      08:16 AM        04:39 PM
2015-01-04     132446           0        -0.2      08:16 AM        04:40 PM
2015-01-05     152611           1        -0.5      08:16 AM        04:41 PM

             count           mean           std       min       25%       50%       75%       max
verbrauch   2557.0  169329.063355  27116.871253  102469.0  147992.0  175584.0  190001.0  231190.0
arbeitstag  2557.0       0.687133      0.463751       0.0       0.0       1.0       1.0       1.0
temperatur  2557.0      11.706492      7.845246     -10.0       5.2      11.2      18.0      32.1


# <font size="7">Feature Engineering</font>

<font size="5">Beim Feature Engineering werden neue Merkmale aus den bestehenden Daten/Merkmalen generiert. Im konkreten Fall wird die Anzahl an Tagesstunden aus dem Sonnenaufgang und Sonnenuntergang berechnet. Sonnenaufgang und Sonnenuntergang werden anschließend aus den Daten entfernt.</font>

In [3]:
#Merkmal für Tagesstunden erzeugen

#Tagesstunden berechnen
df["tagesstunden"] = round((pd.to_timedelta(
    pd.to_datetime(df["sonnenuntergang"]).dt.strftime("%H:%M:%S")).dt.total_seconds() - 
    pd.to_timedelta(pd.to_datetime(df["sonnenaufgang"]).dt.strftime("%H:%M:%S")).dt.total_seconds()) / 3600, 1)

#Sonnenaufgang und Sonnenuntergang werden durch Tagesstunden ersetzt
df.drop(["sonnenaufgang", "sonnenuntergang"], axis=1, inplace=True)

#DataFrame ausgeben
print(df.head())
print()
print(df.describe().transpose())

            verbrauch  arbeitstag  temperatur  tagesstunden
datum                                                      
2015-01-01     126197           0        -2.5           8.4
2015-01-02     147085           1        -0.0           8.4
2015-01-03     141426           0         1.2           8.4
2015-01-04     132446           0        -0.2           8.4
2015-01-05     152611           1        -0.5           8.4

               count           mean           std       min       25%       50%       75%       max
verbrauch     2557.0  169329.063355  27116.871253  102469.0  147992.0  175584.0  190001.0  231190.0
arbeitstag    2557.0       0.687133      0.463751       0.0       0.0       1.0       1.0       1.0
temperatur    2557.0      11.706492      7.845246     -10.0       5.2      11.2      18.0      32.1
tagesstunden  2557.0      12.254634      2.700311       8.3       9.7      12.3      14.9      16.2


# <font size="7">Featurization</font>

<font size="5">Bei der Featurization werden die vorhandenen Merkmale in ein für Algorithmen (besser) zu verarbeitendes Format umgewandelt. Konkret werden die Merkmale normalisiert, also auf eine Skala zwischen 0 und 1 transformiert. Dadurch müssen die Koeffizienten/Gewichte keine allzu großen Werte annehmen, außerdem sind die Abstände zwischen der Werten einzelner Merkmale besser interpretierbar.</font>

<font size="5">Um einen möglichst realistischen Test zu gewährleisten, werden die Scaler zunächst nur mit den Trainingsdaten erstellt, bevor die Daten umgewandelt werden. Ähnlich den Algorithmen sind den Scalern dadurch nur die Trainingsdaten bekannt. Wenn einzelne Beobachtungen außerhalb des Wertebereichs der Trainingsdaten liegen, dann transformiert der Scaler diese Daten (wie im tatsächlichen Einsatz) auf einen Bereich außerhalb von 0 oder 1.</font>

<font size="5">Die Daten werden sowohl skaliert wie auch unskaliert abgespeichert. Die Scaler weden ebenfalls gespeichert.</font>

In [5]:
#Alle Daten speichern
df.to_csv("data.csv")

#Aufteilung in Trainings- und Testdaten
train, test = functions.train_test_split(df, 365)

#Merkmale sklaieren
train_exog = train[["arbeitstag", "temperatur", "tagesstunden"]]
test_exog = test[["arbeitstag", "temperatur", "tagesstunden"]]

scaler_exog = MinMaxScaler(feature_range=(0,1))
scaler_exog.fit(train_exog)

scaled_train_exog = pd.DataFrame(scaler_exog.transform(train_exog), columns=train_exog.columns, index=train_exog.index)
scaled_test_exog = pd.DataFrame(scaler_exog.transform(test_exog), columns=test_exog.columns, index=test_exog.index)

#Ziel skalieren
train_endog = train["verbrauch"]
test_endog = test["verbrauch"]

scaler_endog = MinMaxScaler(feature_range=(0,1))
scaler_endog.fit(train_endog.values.reshape(-1,1))

scaled_train_endog = pd.DataFrame(scaler_endog.transform(train_endog.values.reshape(-1, 1)), columns=["verbrauch"], index=train_endog.index)
scaled_test_endog = pd.DataFrame(scaler_endog.transform(test_endog.values.reshape(-1, 1)), columns=["verbrauch"], index=test_endog.index)

#Ziel und Merkmale zusammenführen
train = scaled_train_endog.join(scaled_train_exog)
test = scaled_test_endog.join(scaled_test_exog)

data_scaled = train.append(test)
data_scaled.to_csv("data_scaled.csv")

In [5]:
#Skalierer speichern
joblib.dump(scaler_exog, "scaler_exog.save")
#scaler_exog = joblib.load("scaler_exog.save")

joblib.dump(scaler_endog, "scaler_endog.save")
#scaler_endog = joblib.load("scaler_endog.save")

['scaler_endog.save']